In [5]:
#!/usr/bin/python3
"""
This program match a project's spectra to the PRIDE Cluster spectral library, to detect the
(low confident) doubted PSMs,
(high confident) approved PSMs,
new PSMs,
and recommend better PSMs for some doubted PSMs.
Usage:
enhancer_analyze.py --project <projectId>
[--host <host_name>]
[--date <date>]
[(--loadfile | --loaddb)]
 enhancer_analyze.py (--help | --version)

Options:
-p, --project=<projectId>            project to be ananlyzed, the files should be putted in this directory
--host=<host_name>                   The host phoenix  to store the data and analyze result
--date =<date>                       The date to specify the tables
--loadfile                           If set, load spectra lib search result from pep.xml file.
--loaddb                             If set, load spectra lib search result from phoenix db.
-h, --help                           Print this help message.
-v, --version                        Print the current version.
"""


import sys, os
import logging
import time,csv
from docopt import docopt
file_dir = "/home/ubuntu/mingze/tools"
sys.path.append(file_dir)
import retrieve_splib_result as retriever
import phoenix_import_util as phoenix
import statistics_util as stat_util
import build_cluster_csv as cluster_csv
import psm_util





    #arguments = docopt(__doc__, version='cluster_phoenix_importer 1.0 BETA')

project_id = 'PXD000222'
host = "localhost"

work_path = '/home/ubuntu/mingze/spec_lib_searching/phospho/'
date = ''
lib_search_results = None
input_path = work_path + project_id + '/'
sr_csv_file = work_path + project_id + '/' + project_id + 'lib_search_result.csv'
try:
    lib_search_results = retriever.retrive_search_result(project_id, input_path, sr_csv_file) #retrieve the library search results and export them to file/phoenix db
except Exception as err:
    print("error in retriving spectraST search result file %s"%(err))

    # export search result to phoenix_db by building the whole big table

psm_file = work_path + project_id + "/" + project_id + "_psm.csv"
print("start to read identification from csv")
identified_spectra  = psm_util.read_identification_from_csv(psm_file)
if identified_spectra == None:
    identified_spectra = phoenix.retrieve_identification_from_phoenix(project_id, host, None)
else:
    print("start to import identification to phoenix db")
    psm_util.insert_psms_to_phoenix_from_csv(project_id, identified_spectra, host)

cluster_data = cluster_csv.read_csv('clusters_min5.csv')
if cluster_data == None:
    cluster_data = phoenix.get_cluster_data(lib_search_results, host)

spec_match_detail_file = work_path + project_id + "/" + project_id + "_spec_match_details.csv"
matched_spec_details_dict = psm_util.read_matched_spec_from_csv(spec_match_detail_file)
if matched_spec_details_dict == None:
    matched_spec_details = psm_util.build_matched_spec(lib_search_results, identified_spectra, cluster_data)
    psm_util.write_matched_spec_to_csv(matched_spec_details, spec_match_detail_file)
    phoenix.upsert_matched_spec_table(project_id, matched_spec_details, host, date)
    matched_spec_details_dict = psm_util.read_matched_spec_from_csv(spec_match_detail_file)

phoenix.create_project_ana_record_table(host)
thresholds = stat_util.default_thresholds
 

start to read identification from csv
start to read identification from csv
start to import identification to phoenix db


In [6]:
len( matched_spec_details_dict)

2898292

In [7]:
len(identified_spectra)

433401

In [9]:
spectra_matched_to_cluster = dict()
unid_spec_matched_to_cluster = dict()  # cluster_id as the key
removed_spec_no = 0
retail_spec_no = 0
psm_dict = matched_spec_details_dict

print("we have %d matched spec"%(len(psm_dict)))
for spec_title in psm_dict.keys():
    psm = psm_dict.get(spec_title)
    f_val = psm.get('f_val', 0)
    cluster_ratio = float(psm.get('cluster_ratio',0.0))
    cluster_size = int(psm.get('cluster_size',0))
    pre_seq = psm.get('pre_seq', '')
    conf_score = float(psm.get('conf_sc'))
    recomm_seq_sc = float(psm.get('recomm_seq_sc'))
    cluster_id = psm.get('cluster_id')

    if f_val < thresholds.get('spectrast_fval_threshold') or \
       cluster_ratio < thresholds.get('cluster_ratio_threshold') or \
       cluster_size < thresholds.get('cluster_size_threshold') :
        removed_spec_no += 1
        continue
    if conf_score and conf_score > 0 \
        and conf_score < thresholds.get('conf_sc_threshold'):  # For pre PSMs with postive confidence score, ignore the PSMs below threshold
        removed_spec_no += 1
        continue
    if conf_score and conf_score < 0 and recomm_seq_sc \
        and recomm_seq_sc < thresholds.get('conf_sc_threshold'):  # For pre PSMs with negtive confidence score or unidentified (-1 for conf_sc), ignore the PSMs whose recommend seq's score is below threshold
        removed_spec_no += 1
        continue
    if pre_seq == None or pre_seq == '':
        if recomm_seq_sc < thresholds.get('conf_sc_threshold'): # For the unidientfied spec, ignore the recommend New PSMs whose recommend seq's score is below threshold
            removed_spec_no += 1
            continue
    matched_spectra = spectra_matched_to_cluster.get(cluster_id, [])
    matched_spectra.append(spec_title)
    spectra_matched_to_cluster[cluster_id] = matched_spectra
    retail_spec_no += 1
print("%d spectra have be filtered by thresholds" %(removed_spec_no))
print("%d spectra have be retailed" %(retail_spec_no))

we have 2898292 matched spec
2615230 spectra have be filtered by thresholds
283062 spectra have be retailed


In [12]:
t_spectra = 0
n_id = 0
n_unid = 0
for cluster_id in spectra_matched_to_cluster.keys():
    matched_spectra = spectra_matched_to_cluster.get(cluster_id, [])
    t_spectra += len(matched_spectra)
    matched_peptides = dict()
    for matched_spec in matched_spectra:
        psm = psm_dict.get(matched_spec, None)
        pre_seq = psm.get('pre_seq')
        pre_mods = psm.get('pre_mods', None)
        if pre_seq:
            n_id += 1
            pep_seq_mods_str = pre_seq
            if pre_mods != None:
                pep_seq_mods_str += "||" + pre_mods
            pep_spectra = matched_peptides.get(pep_seq_mods_str, [])
            pep_spectra.append(matched_spec)
            matched_peptides[pep_seq_mods_str] = pep_spectra
        else:
            n_unid += 1
            matched_unid_spec = unid_spec_matched_to_cluster.get(cluster_id, [])
            matched_unid_spec.append(matched_spec)
            unid_spec_matched_to_cluster[cluster_id] = matched_unid_spec
print("%d id, %d unid"%(n_id, n_unid))
    
print(t_spectra)

217651 id, 65411 unid
283062
